Parâmetros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_parametros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_catalag_schema

Regras e Definições

In [0]:
%sql
select * from tiller.silver.tb_despesas_silver;

In [0]:
set_catalog_and_schema_gold() #Define catalog e schema conforme run

In [0]:
query = f"""
SELECT
    txtCNPJCPF AS cnpj_cpf_fornecedor,
    txtFornecedor AS fornecedor
FROM tiller.silver.tb_despesas_silver
"""

df = spark.sql(query)


df_fornecedores = (
    df
    .groupBy("cnpj_cpf_fornecedor")
    .agg(
        F.first("fornecedor", ignorenulls=True).alias("fornecedor") # Precisei agrupar e pegar o primeiro fornecedor para garantir unicidade por CNPJ/CPF
    )
    .withColumn("data_ingestao", F.date_trunc("second", F.current_timestamp())) #date trunc para considerar somente até os segundos. 
    .withColumn("data_modificacao", F.date_trunc("second", F.current_timestamp()))
)

df_fornecedores.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("tiller.gold.dim_fornecedores")


Comentários das tabelas

In [0]:
%sql
COMMENT ON COLUMN tiller.gold.dim_fornecedores.cnpj_cpf_fornecedor IS 'CNPJ ou CPF do fornecedor';
COMMENT ON COLUMN tiller.gold.dim_fornecedores.fornecedor IS 'Nome do fornecedor';
COMMENT ON COLUMN tiller.gold.dim_fornecedores.data_ingestao IS 'Data e hora da ingestão dos dados na tabela Gold';
COMMENT ON COLUMN tiller.gold.dim_fornecedores.data_modificacao IS 'Data e hora da última modificação registrada na tabela Gold';


In [0]:
%sql

SELECT COUNT(*) AS total, count(distinct cnpj_cpf_fornecedor) as cpf, MIN(data_ingestao) AS primeira_carga
FROM dim_fornecedores;


In [0]:
%sql
select * from tiller.gold.dim_fornecedores;